### Tabular data exploration

- [Parking permits](https://data.somervillema.gov/City-Services/City-of-Somerville-Parking-Permits/xavb-4s9w) between January 1, 2017 and December 31, 2018 
- Registered vehicles - confidential file from Cortni

In [ ]:
# import libraries
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import geopandas
import warnings
warnings.filterwarnings("ignore")

#### 1. Parking Permits

Original file cleanup ->

In [ ]:
# read in full dataset for parking permits 
parking_permits = pd.read_csv('../data/City_of_Somerville_Parking_Permits.csv')

In [ ]:
parking_permits.head()

In [ ]:
# strip extra white space
parking_permits['type_name'] = parking_permits.type_name.str.strip()
parking_permits['st_addr'] = parking_permits.st_addr.str.strip()
parking_permits['st_addr']= parking_permits['st_addr'].apply(lambda mystring: ' '.join(mystring.split()))

# convert issued date to datetime
parking_permits['issued'] = pd.to_datetime(parking_permits['issued'])

Looking at unique types of parking permits - we only care about residential permits - so:

- 'Residential'
- 'Resident - No charge replacement' -- these are replacement permits for those that have been lost, so let's ignore this
- 'New Mass Resident' -- this is a temp permit and valid only for 1 month, so let's get rid of it 

In [ ]:
# parking_permits.type_name.unique()

In [ ]:
print('Size of parking permit data \t\t\t {}\nSize of RESIDENTIAL parking permit data \t {}'.format(
    parking_permits.shape, 
    parking_permits[parking_permits.type_name.isin(
        ['Residential']
    )].shape))

We only care about permits issued in 2018 - so filtering here:


In [ ]:
# parking_permits.issued year is 2018
parking_permits = parking_permits[parking_permits['issued'].dt.year == 2018]

Now make df for Residential permits only ->

In [ ]:
# subset data to only residential parking permits
residential_permits = parking_permits[parking_permits.type_name.isin(
    ['Residential', ])]

In [ ]:
residential_permits.head()

Now aggregate by st address ->

In [ ]:
res_permits_by_st_addr = residential_permits.groupby('st_addr').aggregate({'issued':len}).reset_index()
res_permits_by_st_addr.columns=['st_addr', 'residential_permits_issued']
print('number of unique street addresses: {}'.format(res_permits_by_st_addr.shape[0]))

In [ ]:
res_permits_by_st_addr.head(10)

**Noisy label option 1**  
Number of residential permits issued by street address.

Issues:
- clear inconsistency, such as for 1 Aldersey St, which has 3 garage doors from [google street view](https://www.google.com/maps/place/1+Aldersey+St,+Somerville,+MA+02143/@42.382985,-71.0960374,3a,75y,21.92h,88.7t/data=!3m6!1e1!3m4!1suVgqBBiLUdBI5VRy9pYyYA!2e0!7i16384!8i8192!4m5!3m4!1s0x89e370cca2b22e2d:0x5dbed58b8d9c69f9!8m2!3d42.3830618!4d-71.0958082)
- Data only available for 12617 addresses - need to cross check this

> Can we get info on number of units or whether the house is designated as single or multi-family from Somerville?

#### 2. Registered vehicles
From Cortni: 

> The spreadsheet contains one row per garaged vehicle in the City. Each unique license plate has an anonymized ID (e.g. COS_1). With vehicle registrations, you'll want to make sure you don't double count cars that share a license plate (e.g. Tom had car A for first half of the year, then traded it in for Car B and moved license plate to the new vehicle). In other words, organize data by # of unique license plates per property. Also, keep in mind the caveat we discussed that the addresses on this list are billing addresses, not the garaging address. So you will see some non-Somerville addresses or a car dealership that leases vehicles (excise is billed to dealer who charges lessee). There is also a PDF attached with a key for plate types. 

In [ ]:
registered_vehicles = pd.read_excel('../data/COPY_Registered_Vehicles_16_17.xlsx', sheet_name='raw')
print('full dataset size: ',registered_vehicles.shape)
# remove 2016 registrations - car registrations are valid for two years in MA
registered_vehicles = registered_vehicles[registered_vehicles.Year == 2017]
print('2017 dataset size: ',registered_vehicles.shape)

In [ ]:
# strip extra white space
registered_vehicles['Plate.Type'] = registered_vehicles['Plate.Type'].str.strip()
registered_vehicles['Address'] = registered_vehicles['Address'].str.strip()
registered_vehicles['Address']= registered_vehicles['Address'].apply(lambda mystring: ' '.join(mystring.split()))

In [ ]:
# restrict to plate type categories
categories = ['PAN', 'PAR', 'PAS', 'PAV', 'PAY']
registered_vehicles = registered_vehicles[registered_vehicles['Plate.Type'].isin(categories)]
print('dataset size after restricting to passenger and student vehicles: ', registered_vehicles.shape)
# remove PO box registrations - can't match that up to an address
registered_vehicles = registered_vehicles[~registered_vehicles['Address'].str.contains("PO BOX")]
print('dataset size after removing PO boxes: ', registered_vehicles.shape)
# replace address things like st, rd, ln, etc.
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' STREET',' ST')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' ROAD',' RD')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' LANE',' LN')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' DRIVE',' DR')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' AVE',' AV')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' AVNUE',' AV')
registered_vehicles['Address'] = registered_vehicles['Address'].str.replace(' BLVD',' BLV')

In [ ]:
# ACCOUNT FOR DUPLICATE CARS: 
# group by address, city, unit, plate ID (in case of cars that share a license plate)
count_of_duplicates = registered_vehicles.groupby(['Address', 'City', 'Unit', 'PlateID']).aggregate(
    {'EV':len}).reset_index()
count_of_duplicates.rename(columns={'EV':'duplicate_count'}, inplace=True)
print('accounted for {} duplicates'.format(count_of_duplicates[count_of_duplicates.duplicate_count >1].shape[0]))

In [ ]:
# now get registered vehicles by address
registered_vehicles_by_addr = count_of_duplicates.groupby(
    ['Address', 'City']).aggregate({'PlateID': len}).reset_index()
registered_vehicles_by_addr.rename(columns={'PlateID':'num_registered_vehicles'}, inplace=True)
print('There are {} unique addresses after preliminary data cleaning.\
 (more cleaning to come)'.format(registered_vehicles_by_addr.shape[0]))

>> need to clean this more if we decide 

In [ ]:
registered_vehicles_by_addr.City.unique()

In [ ]:
cities_to_keep = ['SOMERVILLE     ', 
                  'E SOMERVILLE   ',
                  'W SOMERVILLE   '
                 ]
registered_vehicles_by_addr[registered_vehicles_by_addr.City.isin(cities_to_keep)].head(10)


Here are some addresses with a large number of registered vehicles. Most addresses in Somerville correspont to large apartment buildings, while some out of state ones correspond to insurance offices. Note that we have 29 cars with no address listed.

In [ ]:
# here are some interesting ones - where number of registered vehicles is quite large 
## most somerville ones are apartment buildings, but others include a car insurance offie
registered_vehicles_by_addr[registered_vehicles_by_addr.num_registered_vehicles > 20].head()

### 3. Parcel FY19 text data


In [ ]:
parcel_data = pd.read_csv('../data/Parcels_FY19/VisionExtract_FY19.txt', error_bad_lines=False)

In [ ]:
parcel_data.head()

In [ ]:
selected_cols = ['ID',
#  'PROP_ID',
#  'BLDG_VAL',
#  'LAND_VAL',
#  'OTHER_VAL',
#  'TOTAL_VAL',
# #  'FY',
#  'LOT_SIZE',
#  'LS_DATE',
#  'LS_PRICE',
#  'USE_CODE',
 'SITE_ADDR',
 'ADDR_NUM',
 'FULL_STR',
#  'LOCATION',
 'SITE_CITY',
 'SITE_ZIP',
#  'OWNER1',
#  'OWNER2',
#  'OWN_ADDR1',
#  'OWN_ADDR2',
#  'OWN_CITY',
#  'OWN_STATE',
#  'OWN_ZIP',
#  'OWN_CO',
#  'LS_BOOK',
#  'LS_PAGE',
#  'REG_ID',
#  'ZONE',
 'YEAR_BUILT',
 'BLD_AREA',
 'UNITS',
 'RES_AREA',
 'STYLE',
 'STORIES',
 'NUM_ROOMS',
 'LOT_UNITS',
#  'CAMA_ID',
#  'LOC_ID',
#  'MAP',
#  'MAP_CUT',
#  'BLOCK',
#  'BLOCK_CUT',
#  'LOT',
#  'LOT_CUT',
#  'UNIT',
#  'UNIT_CUT',
#  'MBL',
#  'AV PID'
                ]
parcel_data[selected_cols].head(10)

In [ ]:
# parcel_data[parcel_data.STYLE =='Outbuildings']


In [ ]:
keep = [
#     'Office/Apts',
 '2-Decker',
 '3-Decker',
 '3 fam Conv',
 'Two Family',
#  'Vacant Land',
 'Mansard',
#  'Store',
#  'School/College',
 'Two decker',
 'Condominium',
 'Conventional',
 'Family Duplex',
 'Mansard-Apts',
 '2 Fam Conv',
 'Stores/Apt Com',
 'Family Duplex-Apts',
#  'Outbuildings',
 'Mid rise',
 'Two Family-Apts',
#  'Restaurant',
#  'Warehouse',
 'Row Mid',
#  'Office Bldg',
#  'Service Shop',
#  'Research/Devel',
 '3-Decker-Apts',
 'Row End-Apts',
 'Garage/Office',
 'Row End',
 'Row Mid-Apts',
 'Duplex',
 'Fam Conv',
 'Apartments',
 'Victorian',
 'Cottage Bungalow',
 'Conventional-Apts',
 'Double 3D',
 'Three decker',
 'Townhouse end',
 'Townhouse middle',
#  'Retail/Offices',
 'High End Constr',
 '2-Decker-Apts',
 'Convert Warehs/Loft',
#  'Nightclub/Bar',
#  'Clubs/Lodges',
#  'Car Wash',
 'Office/Warehs',
#  'Profess. Bldg',
#  'Hotel',
#  'Truck Terminal',
#  'Pre-Eng Warehs',
#  'Colleges',
 'Dormitory',
#  'Churches',
#  'Telephone Bldg',
 'Indust Condo',
#  'Condo Office',
#  'Supermarkets',
#  'Coin-op CarWsh',
 'Retail Condo',
#  'Fire Station',
#  'Finan Inst.',
#  'Library',
#  'Funeral Home',
 'Low rise',
#  'Other Municip',
#  'Stores/Office',
#  'Bakery',
#  'Dry Cln/Laundr',
#  'Serv Sta 2-bay',
#  'Converted School',
#  'Other State',
#  'Branch Bank',
#  'Theaters Encl.',
 'Mid Rise Apartments',
#  'Light Indust',
 'Cottage',
 'Row Middle',
#  'Serv Sta 3-Bay',
#  'Auto Sales Rpr',
 'Townhouse',
#  'Home for Aged',
#  'Hospitals-Priv',
#  'Commercial Bld',
#  'Skating Arena',
#  'Day Care',
#  'Child Care',
#  'Health Club/Gym',
#  'Supermarket',
#  'City/Town Hall',
#  'Other Federal',
#  'Fast Food Rest',
 'Victorian-Apts',
#  'Converted Municipal',
#  'Comm Warehouse',
#  'Commercial',
#  'Department Str',
 'High Rise Apt',
#  'Shop Center RE',
#  'Food Process',
#  'Hospital',
#  'Schools-Public',
#  'Pkg Garage'
]

In [ ]:
res_types = parcel_data[parcel_data.STYLE.isin(keep)]
res_types.shape

In [ ]:
parcel_data.shape

In [ ]:
parcel_data[parcel_data.ID == 10026]

In [ ]:
parcels = geopandas.read_file('../data/Parcels_FY19')
print('number of parcels: ',parcels.shape[0])

In [ ]:
parcels[parcels.OBJECTID == 10912]